<a href="https://colab.research.google.com/github/farcryson/House_Prices_Prediction/blob/main/Adv_House_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **House Prediction Problem**
**Dataset Link: https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview**

# **Importing** **Libraries**

In [ ]:
!pip install catboost

In [2]:
import pandas as pd
import numpy as np

# **Importing** **Dataset**

In [3]:
dataset_train = pd.read_csv('train.csv')
dataset_test = pd.read_csv('test.csv')

In [ ]:
dataset_train.head()

In [ ]:
dataset_train.info()

# **Data Cleaning**

In [6]:
df_train_clean = dataset_train.copy()
df_train_clean.drop(['Id','SalePrice'], axis=1,inplace=True)

In [7]:
df_test_clean = dataset_test.copy()
df_test_clean.drop('Id',axis=1,inplace=True)

In [8]:
 lim = 0.5*len(df_train_clean)

In [ ]:
lim

# **Imputation(filling null values)**

In [10]:
for col in df_train_clean.columns:
  if(df_train_clean[col].count() < lim):
      df_train_clean.drop(col, axis =1, inplace=True)
      df_test_clean.drop(col, axis =1, inplace=True)

In [ ]:
df_train_clean.info()

In [ ]:
df_test_clean.info()

In [13]:
def fill(data):
  for col in data.columns:
    if data[col].dtype=='int64'or data[col].dtype=='float64':
      data[col].fillna(data[col].mean(), inplace=True)
    elif data[col].dtype=='object':
      data[col].fillna(data[col].mode().iloc[0], inplace=True)

In [14]:
fill(df_train_clean)

In [15]:
fill(df_test_clean)

In [ ]:
df_train_clean.info()

In [ ]:
df_test_clean.info()

# **One Hot Encoding categorical columns**

Concatinating both train and test datasets because some values in columns of train data and test data do not match. By concatinating them we will ensure that there will be accurate one hot encoded columns for both datasets.

In [18]:
mixed_df = pd.concat([df_train_clean, df_test_clean])

In [19]:
cat_col = []
for col in mixed_df.columns:
  if(mixed_df[col].dtype=='object'):
    cat_col.append(col)

In [20]:
df_encoded = pd.DataFrame(pd.get_dummies(mixed_df, columns= cat_col, drop_first=True))

In [21]:
df_encoded.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706.0,0.0,...,0,0,0,0,1,0,0,0,1,0
1,20,80.0,9600,6,8,1976,1976,0.0,978.0,0.0,...,0,0,0,0,1,0,0,0,1,0
2,60,68.0,11250,7,5,2001,2002,162.0,486.0,0.0,...,0,0,0,0,1,0,0,0,1,0
3,70,60.0,9550,7,5,1915,1970,0.0,216.0,0.0,...,0,0,0,0,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655.0,0.0,...,0,0,0,0,1,0,0,0,1,0


Now we divide them into train and test datasets

In [22]:
final_df_train = df_encoded.iloc[:1460,:]
final_df_test = df_encoded.iloc[1460:, :]

In [23]:
final_df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706.0,0.0,...,0,0,0,0,1,0,0,0,1,0
1,20,80.0,9600,6,8,1976,1976,0.0,978.0,0.0,...,0,0,0,0,1,0,0,0,1,0
2,60,68.0,11250,7,5,2001,2002,162.0,486.0,0.0,...,0,0,0,0,1,0,0,0,1,0
3,70,60.0,9550,7,5,1915,1970,0.0,216.0,0.0,...,0,0,0,0,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655.0,0.0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,1999,2000,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,1,0
1456,20,85.0,13175,6,6,1978,1988,119.0,790.0,163.0,...,0,0,0,0,1,0,0,0,1,0
1457,70,66.0,9042,7,9,1941,2006,0.0,275.0,0.0,...,0,0,0,0,1,0,0,0,1,0
1458,20,68.0,9717,5,6,1950,1996,0.0,49.0,1029.0,...,0,0,0,0,1,0,0,0,1,0


In [24]:
final_df_test

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,20,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,...,0,0,0,0,1,0,0,0,1,0
1,20,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,...,0,0,0,0,1,0,0,0,1,0
2,60,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,...,0,0,0,0,1,0,0,0,1,0
3,60,78.0,9978,6,6,1998,1998,20.0,602.0,0.0,...,0,0,0,0,1,0,0,0,1,0
4,120,43.0,5005,8,5,1992,1992,0.0,263.0,0.0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,21.0,1936,4,7,1970,1970,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,1,0
1455,160,21.0,1894,4,5,1970,1970,0.0,252.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1456,20,160.0,20000,5,7,1960,1996,0.0,1224.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1457,85,62.0,10441,5,5,1992,1992,0.0,337.0,0.0,...,0,0,0,0,1,0,0,0,1,0


In [25]:
X = final_df_train.values
y = dataset_train['SalePrice'].values

# **Splitting dataset into training and validation sets**

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

# **Feature Scaling**

In [27]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# **Training the model**

In [28]:
# import xgboost
# regressor = xgboost.XGBRegressor()
# regressor.fit(X_train, y_train)

In [29]:
# from sklearn.tree import DecisionTreeRegressor
# regressor = DecisionTreeRegressor(random_state = 0)
# regressor.fit(X_train, y_train)

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
# regressor.fit(X_train, y_train)

In [ ]:
from catboost import CatBoostRegressor
regressor = CatBoostRegressor()
regressor.fit(X_train, y_train)

# **Predicting Validation set**

In [40]:
y_pred = regressor.predict(X_test)

# **Applying K-fold cross validation**

As cross_val_score doesn't contain "**log root mean squared error**". We create a custom scorer with the help of "**make_scorer**"

In [34]:
def log_RMSE(y_true, y_pred):
    log_y_true = np.log(y_true)
    log_y_pred = np.log(y_pred)
    return ( np.sum( (log_y_true - log_y_pred)**2 ) / len(log_y_true) ) ** 0.5

In [43]:
from sklearn.metrics import make_scorer

scorer = make_scorer(log_RMSE, greater_is_better = False)

In [ ]:
scores = cross_val_score(regressor, X = X_train, y = y_train, cv=10, scoring=scorer)
print("Accuracy: {:.2f} %".format(scores.mean()*100))
print("Standard Deviation: {:.2f} %".format(scores.std()*100))

# **Repeating steps of train dataset**

In [46]:
test_X = final_df_test.values

In [47]:
test_X = sc.transform(test_X)

In [48]:
y_predict = regressor.predict(test_X)

# **Getting the submission file**

In [49]:
temp = pd.read_csv('sample_submission.csv')

In [50]:
pred = pd.DataFrame(y_predict)

In [54]:
output = pd.concat([temp['Id'],pred],axis=1)

In [57]:
output.columns = ['Id', 'SalePrice']

In [58]:
output

,Id,SalePrice
0,1461,121071.767788
1,1462,161707.114643
2,1463,189632.862383
3,1464,190821.617065
4,1465,182684.542123
...,...,...
1454,2915,81748.042226
1455,2916,78632.480086
1456,2917,167878.691305
1457,2918,118855.845885


In [59]:
output.to_csv('output_catboost.csv', index= False)